In [1]:
pip install -q transformers~=4.16.2

     |████████████████████████████████| 3.5 MB 17.4 MB/s 
     |████████████████████████████████| 895 kB 52.4 MB/s 
     |████████████████████████████████| 6.6 MB 62.5 MB/s 
     |████████████████████████████████| 596 kB 50.9 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 


In [2]:
pip install rouge

In [3]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer_1 = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model_1 = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

In [4]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/capstone-29/news-topic/summarization_data/json_to_csv_train_신문기사.csv").drop(['Unnamed: 0'],axis=1)
df

,media,id,article_orginal,abstractive,extractive
0,광양신문,290741778,"['ha당 조사료 400만원…작물별 차등 지원', '이성훈 sinawi@hanmai...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"['8억 투입, 고소천사벽화·자산마을에 색채 입혀', '이성훈 sinawi@hanm...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
2,광양신문,290741793,"['전남드래곤즈 해맞이 다짐…선수 영입 활발', '이성훈 sinawi@hanmail...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,"[3, 5, 7]"
3,광양신문,290741794,"['11~24일, 매실·감·참다래 등 지역특화작목', '이성훈 sinawi@hanm...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","[2, 3, 4]"
4,광양신문,290741797,"['홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항', '이성훈 sinawi@...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"
...,...,...,...,...,...
243978,중도일보,368476513,"['내달 12일까지, 노동인권 감수성 높인다', '당진시는 지난 12일 첫 교육을 ...",당진시는 12일부터 당진시 노동상담소에서지역 청소년들을 대상으로 노동 인권 교육을 ...,"[1, 2, 6]"
243979,중도일보,368476529,"['마을별 10명 이상이면 신청가능, 12월 24일까지 모집', '서산 마을학교 합...",서산시는 배움의 기회를 갖지 못한 어르신을 대상으로 성인문해교실(마을학교)를 운영하...,"[5, 4, 8]"
243980,중도일보,368476533,"['2020년 예산안, 제4회 추경예산안 및 조례안 29건 등 41개 안건 처리 예...",서산시의회는 25일 제1차 본회의를 열고 다음달 13일까지 열리는 제247회 제2차...,"[1, 3, 4]"
243981,중도일보,368479493,"['전담기관 설립 추진, 타당성 검토 용역 착수보고회 개최', '아산시(시장 오세현...",아산시가 25일 시청 상황실에서 전국적 허브를 구축하기 위한 재활헬스케어힐링스파산업...,"[1, 2, 3]"


In [5]:
from tqdm import tqdm

results = []

for i in tqdm(range(50)):
  str_list = df.loc[i]['article_orginal'][2:-2].split("', '")
  text = " ".join(str_list)

  hypothesis = df.loc[i]['abstractive']

  raw_input_ids_1 = tokenizer_1.encode(text)
  input_ids_1 = [tokenizer_1.bos_token_id] + raw_input_ids_1 + [tokenizer_1.eos_token_id]
  summary_ids_1 = model_1.generate(torch.tensor([input_ids_1]), 
                                 length_penalty=2.0,
                                 num_beams=4,  
                                 max_length=1536)
  reference = tokenizer_1.decode(summary_ids_1.squeeze().tolist(), skip_special_tokens=True)

  results.append([hypothesis, reference])

100%|██████████| 50/50 [05:51<00:00,  7.03s/it]


In [6]:
from rouge import Rouge 

hyps_1, refs_1 = map(list, zip(*[[d[0], d[1]] for d in results]))
rouge = Rouge()
scores_1 = rouge.get_scores(hyps_1, refs_1, avg=True)
print(scores_1)

# "f" stands for f1_score, "p" stands for precision, "r" stands for recall.

{'rouge-1': {'r': 0.15513929875750365, 'p': 0.21471721345820283, 'f': 0.17271629662655433}, 'rouge-2': {'r': 0.07792826048984736, 'p': 0.1019494571038314, 'f': 0.08450013006325856}, 'rouge-l': {'r': 0.1478252463048102, 'p': 0.2034556560441686, 'f': 0.16408036783712426}}


In [8]:
scores_12 = rouge.get_scores(hyps_1, refs_1)
print(scores_12)

[{'rouge-1': {'r': 0.5263157894736842, 'p': 0.5714285714285714, 'f': 0.5479452004878964}, 'rouge-2': {'r': 0.4594594594594595, 'p': 0.5, 'f': 0.4788732344455466}, 'rouge-l': {'r': 0.5263157894736842, 'p': 0.5714285714285714, 'f': 0.5479452004878964}}, {'rouge-1': {'r': 0.29411764705882354, 'p': 0.19230769230769232, 'f': 0.23255813475392115}, 'rouge-2': {'r': 0.23529411764705882, 'p': 0.16, 'f': 0.19047618565759653}, 'rouge-l': {'r': 0.29411764705882354, 'p': 0.19230769230769232, 'f': 0.23255813475392115}}, {'rouge-1': {'r': 0.05128205128205128, 'p': 0.11764705882352941, 'f': 0.07142856720025534}, 'rouge-2': {'r': 0.02631578947368421, 'p': 0.0625, 'f': 0.03703703286694149}, 'rouge-l': {'r': 0.05128205128205128, 'p': 0.11764705882352941, 'f': 0.07142856720025534}}, {'rouge-1': {'r': 0.3076923076923077, 'p': 0.3076923076923077, 'f': 0.3076923026923078}, 'rouge-2': {'r': 0.15384615384615385, 'p': 0.16666666666666666, 'f': 0.15999999500800016}, 'rouge-l': {'r': 0.3076923076923077, 'p': 0.30

In [7]:
print(results[0][0])
print(results[0][1])

전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .
전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 일정 소득차를 보전해주는 제도인 '논 타작물 지원사업 추진협의회'를 구성하여 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.


In [9]:
tokenizer_2 = PreTrainedTokenizerFast.from_pretrained('ainize/kobart-news')
model_2 = BartForConditionalGeneration.from_pretrained('ainize/kobart-news')

Downloading:   0%|          | 0.00/302 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

In [12]:
from tqdm import tqdm

results_2 = []

for i in tqdm(range(50)):
  str_list_2 = df.loc[i]['article_orginal'][2:-2].split("', '")
  # print(str_list[-1])
  text_2 = " ".join(str_list_2)
  # print(text)

  hypothesis_2 = df.loc[i]['abstractive']

  input_ids_2 = tokenizer_2.encode(text_2, return_tensors='pt')
  summary_ids_2 = model_2.generate(input_ids=input_ids_2, 
                               length_penalty=2.0, 
                               num_beams=4, 
                               max_length=1536, 
                               eos_token_id=model_2.config.eos_token_id,
                               bos_token_id=model_2.config.bos_token_id)
  reference_2 = tokenizer_2.decode(summary_ids_2[0], skip_special_tokens=True)

  results_2.append([hypothesis_2, reference_2])

100%|██████████| 50/50 [05:03<00:00,  6.07s/it]


In [13]:
hyps_2, refs_2 = map(list, zip(*[[d[0], d[1]] for d in results_2]))
scores_2 = rouge.get_scores(hyps_2, refs_2, avg=True)
print(scores_2)

# "f" stands for f1_score, "p" stands for precision, "r" stands for recall.

{'rouge-1': {'r': 0.17888199512250136, 'p': 0.2756969463096596, 'f': 0.20747595109158712}, 'rouge-2': {'r': 0.09937234055127908, 'p': 0.15066704095527883, 'f': 0.11442094390399662}, 'rouge-l': {'r': 0.1708760277220106, 'p': 0.25672444864908134, 'f': 0.1967554061398167}}


In [14]:
scores_22 = rouge.get_scores(hyps_2, refs_2)
print(scores_22)

[{'rouge-1': {'r': 0.3877551020408163, 'p': 0.5428571428571428, 'f': 0.45238094751984126}, 'rouge-2': {'r': 0.3137254901960784, 'p': 0.47058823529411764, 'f': 0.37647058343529416}, 'rouge-l': {'r': 0.3877551020408163, 'p': 0.5428571428571428, 'f': 0.45238094751984126}}, {'rouge-1': {'r': 0.12, 'p': 0.23076923076923078, 'f': 0.15789473234072035}, 'rouge-2': {'r': 0.08163265306122448, 'p': 0.16, 'f': 0.10810810363403962}, 'rouge-l': {'r': 0.12, 'p': 0.23076923076923078, 'f': 0.15789473234072035}}, {'rouge-1': {'r': 0.045454545454545456, 'p': 0.058823529411764705, 'f': 0.05128204636423453}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.045454545454545456, 'p': 0.058823529411764705, 'f': 0.05128204636423453}}, {'rouge-1': {'r': 0.34615384615384615, 'p': 0.6923076923076923, 'f': 0.4615384570940172}, 'rouge-2': {'r': 0.19230769230769232, 'p': 0.4166666666666667, 'f': 0.2631578904155125}, 'rouge-l': {'r': 0.3076923076923077, 'p': 0.6153846153846154, 'f': 0.4102564058119659}}, 

In [15]:
print(results_2[0][0])
print(results_2[0][1])

전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .
전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 쌀 생산조정제를 적극 추진키로 했으며, 쌀 생산조정제는 벼를 심었던 논에 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도로 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.


In [16]:
tokenizer_3 = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model_3 = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

In [18]:
from tqdm import tqdm

results_3 = []

for i in tqdm(range(50)):
  str_list_3 = df.loc[i]['article_orginal'][2:-2].split("', '")
  # print(str_list[-1])
  text_3 = " ".join(str_list_3)
  # print(text)

  hypothesis_3 = df.loc[i]['abstractive']

  raw_input_ids_3 = tokenizer_3.encode(text_3)
  input_ids_3 = [tokenizer_3.bos_token_id] + raw_input_ids_3 + [tokenizer_3.eos_token_id]
  summary_ids_3 = model_3.generate(torch.tensor([input_ids_3]), 
                               length_penalty=2.0, 
                               num_beams=4, 
                               max_length=1536, 
                               eos_token_id=model_3.config.eos_token_id,
                               bos_token_id=model_3.config.bos_token_id)
  reference_3 = tokenizer_3.decode(summary_ids_3[0], skip_special_tokens=True)

  results_3.append([hypothesis_3, reference_3])

100%|██████████| 50/50 [04:28<00:00,  5.37s/it]


In [19]:
hyps_3, refs_3 = map(list, zip(*[[d[0], d[1]] for d in results_3]))
scores_3 = rouge.get_scores(hyps_3, refs_3, avg=True)
print(scores_3)

# "f" stands for f1_score, "p" stands for precision, "r" stands for recall.

{'rouge-1': {'r': 0.15448338728112376, 'p': 0.23734092159274692, 'f': 0.18008495713486813}, 'rouge-2': {'r': 0.07700168360592177, 'p': 0.11853084897092306, 'f': 0.08948148207128602}, 'rouge-l': {'r': 0.14543786853228022, 'p': 0.22224928158026772, 'f': 0.1693662917127734}}


In [20]:
scores_32 = rouge.get_scores(hyps_3, refs_3)
print(scores_32)

[{'rouge-1': {'r': 0.4883720930232558, 'p': 0.6, 'f': 0.5384615335141354}, 'rouge-2': {'r': 0.4418604651162791, 'p': 0.5588235294117647, 'f': 0.4935064885748019}, 'rouge-l': {'r': 0.4883720930232558, 'p': 0.6, 'f': 0.5384615335141354}}, {'rouge-1': {'r': 0.023255813953488372, 'p': 0.038461538461538464, 'f': 0.028985502549885238}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.023255813953488372, 'p': 0.038461538461538464, 'f': 0.028985502549885238}}, {'rouge-1': {'r': 0.11764705882352941, 'p': 0.11764705882352941, 'f': 0.11764705382352963}, 'rouge-2': {'r': 0.05555555555555555, 'p': 0.0625, 'f': 0.05882352442906617}, 'rouge-l': {'r': 0.11764705882352941, 'p': 0.11764705882352941, 'f': 0.11764705382352963}}, {'rouge-1': {'r': 0.4166666666666667, 'p': 0.7692307692307693, 'f': 0.5405405359824691}, 'rouge-2': {'r': 0.25, 'p': 0.5, 'f': 0.33333332888888895}, 'rouge-l': {'r': 0.375, 'p': 0.6923076923076923, 'f': 0.4864864819284149}}, {'rouge-1': {'r': 0.027777777777777776, 'p'

In [21]:
print(results_3[0][0])
print(results_3[0][1])

전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .
전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 쌀 생산조정제를 적극 추진키로 했으며, 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급 불안 품목은 제외될 예정이다.
